In [ ]:
from pathlib import Path

import SimpleITK as sitk
import numpy as np
from skimage.measure import regionprops, label

In [ ]:
def get_path_and_patients(path):
    """Obtain path for data, with all patient (numbers) from the data directory"""

    absolute_path = Path().absolute().parent
    data_path = absolute_path / path
    patients = [obs.name for obs in data_path.iterdir()]
    return data_path, patients


def get_array(nii_path):
    """Obtain numpy array data from .nii.gz file"""

    nifti = sitk.ReadImage(nii_path)
    array = sitk.GetArrayFromImage(nifti)
    return array


def get_image_and_mask(path, observation, task):
    """Obtain the 3D image and the corresponding mask"""

    task_name = pre if task == 1 else mid
    obs_path = path / observation / task_name
    if task_name == pre:
        image_path = str(list(obs_path.glob("*T2*"))[0])
        mask_path = str(list(obs_path.glob("*mask*"))[0])
    if task_name == mid:
        image_path = str(list(obs_path.glob("*midRT_T2*"))[0])
        mask_path = str(list(obs_path.glob("*midRT_mask*"))[0])
    print(image_path)
    print(mask_path)
    img = get_array(image_path)
    mask = get_array(mask_path)

    return img, mask


def get_objects(mask, label_num):
    """Obtain objects from label image from corresponding label number"""

    label_img = label(mask == label_num)
    regions = regionprops(label_img)
    return regions


def number_of_objects(mask, label_num):
    """Count the number of objects of corresponding label number"""

    regions = get_objects(mask, label_num)
    n_objects = len(regions)
    return n_objects

In [ ]:
pre = 'preRT'
mid = 'midRT'

In [ ]:
dir_path = r"data\HNTSMRG24_train"

# TASK 1

In [ ]:
task = 1
path, patients = get_path_and_patients(dir_path)
len(patients)

In [ ]:
GTVp_t1 = []
GTVn_t1 = []
for i, patient in enumerate(patients):
    print(f'Patient: {patient} -- N{i:03d}')
    img, mask = get_image_and_mask(path, patient, task)
    GTVp_t1.append([region.area for region in get_objects(mask, 1)])
    GTVn_t1.append([region.area for region in get_objects(mask, 2)])


# Task 2

In [ ]:
path

In [ ]:
task = 2
path, patients = get_path_and_patients(dir_path)
len(patients)

In [ ]:
GTVp_t2 = []
GTVn_t2 = []
for i, patient in enumerate(patients):
    print(f'Patient: {patient} -- N{i:03d}')
    img, mask = get_image_and_mask(path, patient, task)
    GTVp_t2.append([region.area for region in get_objects(mask, 1)])
    GTVn_t2.append([region.area for region in get_objects(mask, 2)])


In [ ]:
def count(GTVlist):
    lens = [len(l) for l in GTVlist]
    vlist = [v for l in GTVlist for v in l]
    print(f'Mean voxels: {np.mean(vlist)}\nNum volumes: {np.sum(lens)}\nNum nonpresent: {lens.count(0)}')

In [ ]:
count(GTVp_t1)

In [ ]:
count(GTVp_t2)

In [ ]:
count(GTVn_t1)

In [ ]:
count(GTVn_t2)